In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from google.colab import files

# Abre a janela de upload para o professor subir o CSV manualmente
uploaded = files.upload()

# Depois de fazer o upload, usa o nome do arquivo diretamente
import pandas as pd

df = pd.read_csv("vendas_modificado.csv")


Saving vendas_modificado.csv to vendas_modificado (1).csv


In [23]:
import unicodedata

In [24]:
df.head()

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,"R$ 16,87",13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.00,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,"R$ 3,25",3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.00,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,"R$ 1,63",9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.00,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,Carvão,"R$ 8,74",4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.00,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,"R$ 9,48",2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.00,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações


In [25]:
for col in ['valor', 'total']:
    df[col] = df[col].astype(str).str.strip()

    # Remover o símbolo 'R$', mantendo espaço para evitar erros
    df[col] = df[col].str.replace(r'R\$\s*', '', regex=True)

    # Substituir a vírgula por ponto para os valores decimais
    df[col] = df[col].str.replace(r',', '.', regex=True)

    # Remover pontos de milhar (apenas se houver separadores de milhar válidos)
    df[col] = df[col].str.replace(r'(?<=\d)(?=(\d{3})+(?!\d))', '', regex=True)

    # Substituir strings vazias por NaN para evitar conversões erradas
    df[col] = df[col].replace('', np.nan)

    # Converter para float, tratando erros
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Verificar os valores corrigidos
print("Valores corrigidos na coluna 'valor':")
print(df[['valor']].head())

# Arredondar para 2 casas decimais após a conversão correta
df['valor'] = df['valor'].apply(lambda x: round(x, 2) if pd.notna(x) else np.nan)
df.head()

Valores corrigidos na coluna 'valor':
   valor
0  16.87
1   3.25
2   1.63
3   8.74
4   9.48


,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,16.87,13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.00,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,3.25,3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.00,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,1.63,9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.00,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,Carvão,8.74,4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.00,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,9.48,2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.00,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações


In [26]:
# Garantir que os valores não apareçam em notação científica
pd.set_option('display.float_format', '{:.2f}'.format)

print("Valores na coluna 'valor' após arredondamento:")
print(df[['valor']].head())

# Converter colunas numéricas corretamente
df['quantidade'] = pd.to_numeric(df['quantidade'], errors='coerce').fillna(0).astype(int)
df['frete'] = pd.to_numeric(df['frete'], errors='coerce')  # Converte e mantém NaN onde houver erro

# Remover espaços extras nos nomes das colunas
df.columns = df.columns.str.strip()

# Garantir que as strings não sejam truncadas
colunas_texto = [
    "cliente", "produto", "status", "cidade", "estado", "pais",
    "pagamento", "vendedor", "marca"
]

# Aplique a conversão apenas nas colunas de texto (sem encoding adicional)
for col in colunas_texto:
    df[col] = df[col].str.strip()  # Remover espaços extras
    df[col] = df[col].str.encode('utf-8', errors='ignore').str.decode('utf-8')

df.head()


Valores na coluna 'valor' após arredondamento:
   valor
0  16.87
1   3.25
2   1.63
3   8.74
4   9.48


,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,16.87,13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.00,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,3.25,3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.00,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Água Mineral,1.63,9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.00,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,Carvão,8.74,4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.00,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,9.48,2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.00,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações


In [27]:
# Substitui 'Ã' por 'A' nas colunas de texto
for coluna in colunas_texto:
    df[coluna] = df[coluna].str.replace('Ã', 'A', regex=False)

# Substitui 'Á' por 'A' nas colunas de texto
for coluna in colunas_texto:
    df[coluna] = df[coluna].str.replace('Á', 'A', regex=False)


In [28]:
#EXERCICIO 01

# Padronizar datas e horas
df['data'] = pd.to_datetime(df['data'], errors='coerce')
df['hora'] = pd.to_datetime(df['hora'], format='%H:%M:%S', errors='coerce').dt.time

# Exemplo para verificar as primeiras linhas do DataFrame após ajustes
print(df[['valor']].head())
df.head()

   valor
0  16.87
1   3.25
2   1.63
3   8.74
4   9.48


,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,16.87,13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.00,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,3.25,3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.00,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Agua Mineral,1.63,9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.00,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,Carvão,8.74,4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.00,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,9.48,2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.00,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações


In [29]:
# Conversão de colunas numéricas
df['valor'] = pd.to_numeric(df['valor'], errors='coerce')
df['total'] = pd.to_numeric(df['total'], errors='coerce')
df['quantidade'] = pd.to_numeric(df['quantidade'], errors='coerce')
df['frete'] = pd.to_numeric(df['frete'], errors='coerce')

In [30]:
# EXERCÍCIO 02

# Tratamento de Valores ausentes
df.isnull().sum()  # Verifica quais colunas têm valores ausentes
df.dropna()  # ou usar fillna() se quiser preencher com média, zero, etc.

# Remove dados duplicados
df = df.drop_duplicates()

In [31]:
# Converter 'total' para numérico novamente, garantindo que valores inválidos sejam tratados como NaN
df['total'] = pd.to_numeric(df['total'], errors='coerce')

# Recalcular o total com base no valor e quantidade + frete, arredondado para 2 casas decimais
df['total'] = ((df['valor'] * df['quantidade']).round(2)) + df['frete']

# Garantir que os valores na coluna 'total' também estão com apenas 2 casas decimais
df['total'] = df['total'].round(2)

# Exibir as primeiras linhas para conferir
print("Primeiras linhas do DataFrame após recalcular e corrigir o total:")
df.head()


Primeiras linhas do DataFrame após recalcular e corrigir o total:


<ipython-input-31-ab7bf2ae0568>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total'] = pd.to_numeric(df['total'], errors='coerce')
<ipython-input-31-ab7bf2ae0568>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['total'] = ((df['valor'] * df['quantidade']).round(2)) + df['frete']
<ipython-input-31-ab7bf2ae0568>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS Araujo Kuhn,Queijo Mussarela,16.87,13,239.31,Pagamento Confirmado,Niterói,RJ,Brasil,24000-000,20.00,Cartão de Crédito,SAMUEL HENRIQUE CAÇADOR,Porto Alegre
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,Molho de Tomate,3.25,3,9.75,Pagamento Confirmado,Mariana,MG,Brasil,35420-000,0.00,Pix,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,Fugini
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,Agua Mineral,1.63,9,36.67,Pagamento Confirmado,Cabo Frio,RJ,Brasil,28900-000,22.00,Transferência Bancária,HENRICO MATOS LIMA DA CUNHA,Minalba
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,Carvão,8.74,4,54.96,Em Separação,Campos dos Goytacazes,RJ,Brasil,28000-000,20.00,Pix,GABRIEL QUEIROZ DE AGUIAR,Marca-Brás
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,Café,9.48,2,18.96,Em Separação,Conselheiro Lafaiete,MG,Brasil,36400-000,0.00,Cartão de Crédito,HENRICO MATOS LIMA DA CUNHA,3 Corações


In [32]:
# Padronizar textos (tudo para maiúsculas e sem espaços extras)
colunas_texto = ['cliente', 'produto', 'status', 'cidade', 'estado', 'pais', 'pagamento', 'vendedor', 'marca']
for coluna in colunas_texto:
    df[coluna] = df[coluna].astype(str).str.upper().str.strip()

# Preencher valores NaN em 'frete' com 0.0
df['frete'] = df['frete'].fillna(0.0)

# Padronizar CEPs para 8 dígitos
df['cep'] = df['cep'].astype(str).str.zfill(8)

# Mostrar as primeiras linhas do dataset limpo
df.head()


,id_da_compra,data,hora,cliente,produto,valor,quantidade,total,status,cidade,estado,pais,cep,frete,pagamento,vendedor,marca
0,13679,2021-03-20,23:35:51,LUCAS ARAUJO KUHN,QUEIJO MUSSARELA,16.87,13,239.31,PAGAMENTO CONFIRMADO,NITERÓI,RJ,BRASIL,24000-000,20.00,CARTÃO DE CRÉDITO,SAMUEL HENRIQUE CAÇADOR,PORTO ALEGRE
1,28070,2020-10-30,09:00:53,MICAEL SOUZA RONCETE,MOLHO DE TOMATE,3.25,3,9.75,PAGAMENTO CONFIRMADO,MARIANA,MG,BRASIL,35420-000,0.00,PIX,MICAEL MALAQUIAS DE SOUZA OLIVEIRA,FUGINI
2,47484,2021-06-09,15:30:28,FELIPE AUGUSTO NERY SILVA,AGUA MINERAL,1.63,9,36.67,PAGAMENTO CONFIRMADO,CABO FRIO,RJ,BRASIL,28900-000,22.00,TRANSFERÊNCIA BANCÁRIA,HENRICO MATOS LIMA DA CUNHA,MINALBA
3,20809,2022-06-04,08:41:23,LEVI RIBEIRO AMORIM,CARVÃO,8.74,4,54.96,EM SEPARAÇÃO,CAMPOS DOS GOYTACAZES,RJ,BRASIL,28000-000,20.00,PIX,GABRIEL QUEIROZ DE AGUIAR,MARCA-BRÁS
4,47123,2019-05-04,13:38:45,GABRIEL MATOS LIMA DA CUNHA,CAFÉ,9.48,2,18.96,EM SEPARAÇÃO,CONSELHEIRO LAFAIETE,MG,BRASIL,36400-000,0.00,CARTÃO DE CRÉDITO,HENRICO MATOS LIMA DA CUNHA,3 CORAÇÕES


# **ANÁLISE DE DADOS**

In [33]:
# Produto mais caro
produto_mais_caro = df.loc[df['valor'].idxmax(), 'produto']
valor_mais_caro = df['valor'].max()
print(f"Produto mais caro: {produto_mais_caro}, Valor: R${valor_mais_caro}")

# Cliente com maior valor de compra
cliente_com_maior_valor = df.groupby('cliente')['valor'].sum().idxmax()
valor_total_cliente = df.groupby('cliente')['valor'].sum().max()
print(f"Cliente com maior valor de compra: {cliente_com_maior_valor}, Total gasto: R${valor_total_cliente}")

# Produto mais vendido
produto_mais_vendido = df.groupby('produto')['quantidade'].sum().idxmax()
quantidade_produto_mais_vendido = df.groupby('produto')['quantidade'].sum().max()
print(f"Produto mais vendido: {produto_mais_vendido}, Quantidade vendida: {quantidade_produto_mais_vendido}")


Produto mais caro: VINHO, Valor: R$5670.99
Cliente com maior valor de compra: VICTOR HENRIQUE CAÇADOR, Total gasto: R$64554.75
Produto mais vendido: PASTA DE DENTE, Quantidade vendida: 202660


In [34]:
# Forma de pagamento mais utilizada
forma_pagamento_mais_usada = df['pagamento'].mode()[0]
print(f"Forma de pagamento mais utilizada: {forma_pagamento_mais_usada}")

# Cidade com maior valor de vendas
cidade_mais_vendida = df.groupby('cidade')['valor'].sum().idxmax()
valor_total_cidade = df.groupby('cidade')['valor'].sum().max()
print(f"Cidade com maior valor de vendas: {cidade_mais_vendida}, Total vendido: R${valor_total_cidade}")

Forma de pagamento mais utilizada: CARTÃO DE CRÉDITO
Cidade com maior valor de vendas: BARBACENA, Total vendido: R$179630.08


In [35]:
#ESTÁ ENCONTRANDO OS DUPLICADOS MAS COMOS SABER SE É REDUNDANTE ?#

contagem_ids = df['id_da_compra'].value_counts()

# Filtrar apenas os IDs que aparecem mais de uma vez
ids_duplicados = contagem_ids[contagem_ids > 1].index.tolist()

# Filtrar o DataFrame apenas com os IDs duplicados
df_duplicados = df[df['id_da_compra'].isin(ids_duplicados)]

# Verificar se os valores totais também são os mesmos para cada ID duplicado
duplicados_mesmo_valor = df_duplicados.groupby('id_da_compra')['total'].nunique()

# Filtrar apenas os IDs em que o valor_total também é o mesmo (ou seja, há apenas um valor distinto)
ids_confirmados = duplicados_mesmo_valor[duplicados_mesmo_valor == 1].index.tolist()

# Exibir os IDs duplicados com o mesmo valor total
print("IDs duplicados com mesmo valor total da compra:", ids_confirmados)

IDs duplicados com mesmo valor total da compra: [514, 1984, 2306, 3747, 3784, 3932, 4056, 4126, 4677, 4893, 4969, 6718, 6843, 7594, 7700, 8327, 10957, 11196, 11371, 12589, 12621, 14239, 16184, 16839, 16937, 17505, 17915, 18302, 18710, 18865, 19398, 19713, 19849, 19932, 20084, 20200, 21206, 21538, 22487, 26292, 27929, 28015, 28598, 28982, 30002, 30779, 31030, 32132, 32184, 32565, 32633, 33212, 33224, 33330, 34141, 34369, 34372, 34568, 36359, 36448, 37248, 37262, 37770, 38473, 39873, 40597, 40651, 41687, 42616, 42664, 43402, 44053, 45323, 45411, 45786, 46064, 46101, 46463, 47110, 47567, 48576, 50383, 50415, 51255, 51736, 52442, 52564, 52848, 52972, 53425, 54132, 55244, 55507, 55810, 56759, 56936, 57330, 59036, 59214, 59918]


## **Relatório Final**

- Padronizadas colunas de valores monetários;
- Padronizado strings com acentuação e pontuação;
- Corrigido o formato de datas e horas;
- Removidos valores nulos nas colunas;
- Removidas duplicatas;
- Análise breve sobre o item, forma de pagamento e cidade com maior valor de venda
- Conferência da multiplicação na coluna `total`.